In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
# df_train = df_train.iloc[:-686]

In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

25

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
# df_latest[df_latest['player'] == 'Chris Wood']
# lucas digne
# chris wood

In [10]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [11]:
last_play_proba.head()

,player,team,play_proba
0,Edinson Cavani,Manchester United,1.00
1,David de Gea,Manchester United,1.00
2,Nemanja Matic,Manchester United,0.25
3,Paul Pogba,Manchester United,1.00
4,Harry Maguire,Manchester United,1.00


In [12]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [13]:
df_latest.drop_duplicates(inplace=True)

In [14]:
df_train.drop_duplicates(inplace=True)

In [15]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [16]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [17]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [18]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
Bernardo Silva
João Cancelo
Rúben Dias
Rodri
Aymeric Laporte
Raheem Sterling
Gabriel Jesus
Kyle Walker
Jack Grealish
Phil Foden
Kevin De Bruyne
İlkay Gündoğan
Riyad Mahrez
Nathan Aké
Fernandinho
Oleksandr Zinchenko
John Stones
Ferrán Torres
Cole Palmer
Zack Steffen
Benjamin Mendy
James Mcatee
Liam Delap
Kayky Chagas
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Alisson
Virgil van Dijk
Trent Alexander-Arnold
Mohamed Salah
Sadio Mané
Joël Matip
Diogo Jota
Jordan Henderson
Andrew Robertson
Fabinho
Thiago Alcántara
Roberto Firmino
Alex Oxlade-Chamberlain
Ibrahima Konaté
Naby Keïta
James Milner
Kostas Tsimikas
Curtis Jones
Harvey Elliott
Caoimhín Kelleher
Luis Díaz
Tyler Morton
Takumi Minamino
Divock Origi
Joe Gomez
Kaide Gordon
Neco Williams
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Antonio Rüdiger
Edouard Mendy
Thi

Francisco Sierralta
Edo Kayembe
Ken Sema
Oghenekaro Etebo
Ozan Tufan
Christian Kabasele
Nicolas Nkoulou
Ashley Fletcher
Troy Deeney
Dan Gosling
Burnley
https://fbref.com/en/squads/943e8050/Burnley-Stats
Dwight McNeil
James Tarkowski
Nick Pope
Ashley Westwood
Ben Mee
Matthew Lowton
Chris Wood
Josh Brownhill
Charlie Taylor
Jóhann Berg Guðmundsson
Maxwel Cornet
Jack Cork
Aaron Lennon
Jay Rodriguez
Erik Pieters
Nathan Collins
Ashley Barnes
Connor Roberts
Matěj Vydra
Wout Weghorst
Wayne Hennessey
Dale Stephens


In [19]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [20]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [21]:
df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['round'].str[-2:] != 'on'].index]

In [22]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [23]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [24]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6600,2021-2022,Wout Weghorst,2022-02-13,Burnley,Premier League,25,3,1,36,30,...,0.0,3,0,11,26,42.3,18,0,0,Wout Weghorst
6601,2021-2022,Wayne Hennessey,2021-12-30,Burnley,Premier League,20,0,0,36,0,...,0.1,3,0,18,32,56.3,11,0,0,Wayne Hennessey
6602,2021-2022,Wayne Hennessey,2022-01-02,Burnley,Premier League,21,0,0,51,1,...,0.0,0,0,22,48,45.8,15,0,0,Wayne Hennessey
6603,2021-2022,Dale Stephens,2021-12-30,Burnley,Premier League,20,0,0,20,9,...,0.0,0,0,12,16,75.0,9,0,0,Dale Stephens


In [25]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [26]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

['Kayky Chagas']

In [31]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [27]:
# any additional corrections
# match_list['Kayky Chagas'] = 'Kayky da Silva Chagas'
# match_list['Josh Dasilva'] = 'Pelenda Joshua Dasilva'
# match_list['Bruno Guimarães'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Samir Santos'] = 'Samir Caetano de Souza Santos'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

In [28]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [29]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

[]

In [30]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [33]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [34]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [35]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [36]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [37]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [38]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [39]:
# add crowds field
df_latest_further['crowds'] = 1

In [40]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [41]:
df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds


In [42]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6600,2021-2022,Wout Weghorst,2022-02-13,Burnley,Premier League,25,3,1,36,30,...,0.0,3,0,11,26,42.3,18,0,0,Wout Weghorst
6601,2021-2022,Wayne Hennessey,2021-12-30,Burnley,Premier League,20,0,0,36,0,...,0.1,3,0,18,32,56.3,11,0,0,Wayne Hennessey
6602,2021-2022,Wayne Hennessey,2022-01-02,Burnley,Premier League,21,0,0,51,1,...,0.0,0,0,22,48,45.8,15,0,0,Wayne Hennessey
6603,2021-2022,Dale Stephens,2021-12-30,Burnley,Premier League,20,0,0,20,9,...,0.0,0,0,12,16,75.0,9,0,0,Dale Stephens


In [43]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [44]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
126346,Harry Kane,21,4,90,Tottenham Hotspur,Watford,1.467118,0.338504,False,2,...,0.0,3.0,0.0,24.0,33.0,72.7,38.0,0.0,1.0,1
127048,Harry Kane,22,4,90,Tottenham Hotspur,Leicester City,1.466032,1.169548,False,11,...,0.4,11.0,2.0,25.0,36.0,69.4,49.0,4.0,4.0,1
127815,Harry Kane,23,4,90,Tottenham Hotspur,Chelsea,NaN,NaN,False,2,...,0.2,2.0,0.0,13.0,21.0,61.9,25.0,2.0,2.0,1
128541,Harry Kane,24,4,90,Tottenham Hotspur,Southampton,NaN,NaN,True,2,...,0.5,4.0,1.0,18.0,30.0,60.0,20.0,2.0,2.0,1
129202,Harry Kane,25,4,90,Tottenham Hotspur,Wolverhampton Wanderers,NaN,NaN,True,2,...,0.1,2,0,16,26,61.5,33,2,4,1


In [45]:
df_train_new.tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
129649,Yan Valery,25,2,0,Southampton,Manchester United,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
129650,Yerry Mina,25,2,0,Everton,Leeds,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
129651,Yerson Mosquera Valdelamar,25,2,0,Wolverhampton Wanderers,Tottenham Hotspur,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
129652,Yoane Wissa,25,3,81,Brentford,Crystal Palace,NaN,NaN,True,3,...,0.0,0,0,5,9,55.6,7,0,0,1
129653,Youri Tielemans,25,3,89,Leicester City,West Ham United,NaN,NaN,True,9,...,0.1,4,1,63,72,87.5,47,1,1,1
129654,Yves Bissouma,25,3,26,Brighton and Hove Albion,Watford,NaN,NaN,False,1,...,0.0,0,0,15,17,88.2,16,0,0,1
129655,Yves Bissouma,25,3,90,Brighton and Hove Albion,Manchester United,NaN,NaN,False,2,...,0.0,0,0,53,58,91.4,51,1,3,1
129656,Zack Steffen,25,1,0,Manchester City,Norwich,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
129657,Álvaro Fernández,25,1,0,Brentford,Crystal Palace,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
129658,Çaglar Söyüncü,25,2,90,Leicester City,West Ham United,NaN,NaN,True,1,...,0.0,0,0,64,83,77.1,63,0,0,1


In [46]:
df_latest_further.shape

(700, 57)

In [47]:
df_train_new.shape

(129659, 59)

In [48]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [49]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [50]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
244,West Ham United,Newcastle United,2022-02-19T12:30:00Z,26.0,0
257,Arsenal,Wolverhampton Wanderers,2022-02-24T19:45:00Z,26.0,1
256,Liverpool,Leeds,2022-02-23T19:45:00Z,26.0,2
255,Watford,Crystal Palace,2022-02-23T19:30:00Z,26.0,3
254,Burnley,Tottenham Hotspur,2022-02-23T19:30:00Z,26.0,4
253,Wolverhampton Wanderers,Leicester City,2022-02-20T16:30:00Z,26.0,5
251,Manchester City,Tottenham Hotspur,2022-02-19T17:30:00Z,26.0,6
252,Leeds,Manchester United,2022-02-20T14:00:00Z,26.0,7
249,Liverpool,Norwich,2022-02-19T15:00:00Z,26.0,8
248,Crystal Palace,Chelsea,2022-02-19T15:00:00Z,26.0,9


In [51]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [52]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,21,4
1,26.0,1,Arsenal,Wolverhampton Wanderers,2022-02-24T19:45:00Z,3,39
2,26.0,2,Liverpool,Leeds,2022-02-23T19:45:00Z,14,2
3,26.0,3,Watford,Crystal Palace,2022-02-23T19:30:00Z,57,31
4,26.0,4,Burnley,Tottenham Hotspur,2022-02-23T19:30:00Z,90,6


In [53]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [54]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
3,Pierre-Emerick Aubameyang,0,4,4,96,3,Aubameyang
7,Pablo Marí,0,2,8,42,3,Marí
8,Héctor Bellerín,0,2,9,48,3,Bellerín
...,...,...,...,...,...,...,...
697,Christian Marques,0,2,549,40,39,Marques
699,Louie Moulden,0,1,611,40,39,Moulden
702,Bruno André Cavaco Jordão,0,3,661,45,39,Jordão
704,Dion Sanderson,0,2,686,40,39,Sanderson


In [55]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [56]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [57]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,26.0,0,Newcastle United,West Ham United,2022-02-19T12:30:00Z,Joseph Willock,3,57,100,Willock,False
1,26.0,0,Newcastle United,West Ham United,2022-02-19T12:30:00Z,Matt Targett,2,47,100,Targett,False
2,26.0,0,Newcastle United,West Ham United,2022-02-19T12:30:00Z,Dan Burn,2,44,100,Burn,False
3,26.0,0,Newcastle United,West Ham United,2022-02-19T12:30:00Z,Chris Wood,4,67,100,Wood,False
4,26.0,0,Newcastle United,West Ham United,2022-02-19T12:30:00Z,Jonjo Shelvey,3,53,100,Shelvey,False


In [58]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Kurt Zouma,2,54,75,Zouma,True
1,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Mark Noble,3,44,100,Noble,True
2,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Darren Randolph,1,43,None,Randolph,True
3,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Lukasz Fabianski,1,50,100,Fabianski,True
4,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Angelo Ogbonna,2,49,0,Ogbonna,True


In [59]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [60]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [61]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Kurt Zouma,2,5.4,0.75,Zouma,True,NaN,NaN,2122
1,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Mark Noble,3,4.4,1.00,Noble,True,NaN,NaN,2122
2,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Darren Randolph,1,4.3,1.00,Randolph,True,NaN,NaN,2122
3,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Lukasz Fabianski,1,5.0,1.00,Fabianski,True,NaN,NaN,2122
4,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Angelo Ogbonna,2,4.9,0.00,Ogbonna,True,NaN,NaN,2122
5,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Winston Reid,2,3.9,0.00,Reid,True,NaN,NaN,2122
6,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Aaron Cresswell,2,5.3,1.00,Cresswell,True,NaN,NaN,2122
7,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Andriy Yarmolenko,3,5.1,1.00,Yarmolenko,True,NaN,NaN,2122
8,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Michail Antonio,4,7.7,1.00,Antonio,True,NaN,NaN,2122
9,26.0,0,West Ham United,Newcastle United,2022-02-19T12:30:00Z,Craig Dawson,2,4.9,1.00,Dawson,True,NaN,NaN,2122


In [62]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')